In [ ]:
import os
import torch
from torchvision.utils import save_image
from torch.autograd import Variable
import numpy as np
import torch.nn as nn

# 設定參數
latent_dim = 100  # 潛在向量的維度
img_size = 64     # 圖片大小
channels = 3      # 圖片通道數 (RGB)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.init_size = img_size // 4
        self.fc = nn.Linear(latent_dim, 128 * self.init_size ** 2)
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise):
        out = self.fc(noise)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

# 加載訓練好的生成器模型
generator = Generator()
generator.load_state_dict(torch.load("/home/yuchi/AI/ACGAN/model/generator_25.pth"))  # 替換為最後保存的生成器模型檔案
generator.eval()  # 設定為推理模式

cuda = False
device = "cpu"

# 開始生成圖片
num_images = 500  # 生成圖片數量
batch_size = 16   # 每批生成圖片數量
total_batches = num_images // batch_size

with torch.no_grad():  # 禁用梯度計算以提高效率
    img_counter = 1  # 從 1 開始命名圖片
    for batch in range(total_batches):
        # 隨機生成潛在向量 z
        z = torch.randn(batch_size, latent_dim, device=device)
        
        # 使用生成器生成圖片
        gen_imgs = generator(z)
        
        # 儲存圖片
        for i in range(batch_size):
            save_path = os.path.join(f"/home/yuchi/AI/ACGAN/Result/{img_counter}.jpg")
            save_image(gen_imgs[i], save_path, normalize=True)
            img_counter += 1

# 處理剩餘的圖片（如果 num_images 不是 batch_size 的整數倍）
remaining = num_images % batch_size
if remaining > 0:
    z = torch.randn(remaining, latent_dim, device=device)
    gen_imgs = generator(z)
    for i in range(remaining):
        save_path = os.path.join(f"/home/yuchi/AI/ACGAN/Result/{img_counter}.jpg")
        save_image(gen_imgs[i], save_path, normalize=True)
        img_counter += 1

!python -m pytorch_fid /home/yuchi/AI/anim /home/yuchi/AI/ACGAN/Result --batch-size 16

100%|███████████████████████████████████████████| 32/32 [00:17<00:00,  1.80it/s]
FID:  117.68326672888227
